In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
hoy = datetime.datetime.today()

In [3]:
tr = pd.read_excel('transito_raw.xlsx')

In [4]:
#crear lt fact y total
condicion = [tr['Tipo.pedido']=='S',tr['Tipo.pedido']=='U',tr['Tipo.pedido']=='V']
resultado = [30,10,5]
resultado_2 = [90,25,15]

tr['LT Fact'] = np.select(condicion,resultado,0)
tr['LT Total'] = np.select(condicion,resultado_2,0)



In [5]:
tr['Fecha.creación'] = pd.to_datetime(tr['Fecha.creación'])

In [ ]:
tr['Fecha O'] = np.where(
    tr['Status.línea.pedido'] == 'O',
    np.where(
        tr['Fecha.creación'] + pd.to_timedelta(tr['LT Fact'], unit='D') >= pd.to_datetime(hoy),
        tr['Fecha.creación'] + pd.to_timedelta(tr['LT Total'], unit='D'),
        pd.to_datetime(hoy) + pd.to_timedelta(tr['LT Total'] * 1.5, unit='D')
    ),
    pd.NaT  # Reemplaza 0 con un valor de fecha/tiempo nulo
)

In [7]:
tr['Fecha O'] = pd.to_datetime(tr['Fecha O'])

In [8]:
etas = pd.read_excel('etas.xlsx')

In [9]:
# 1. Convert all date-like columns to datetime format.
#    The `errors='coerce'` will turn any invalid values (like '0') into NaT.
etas['Fecha Arribo Bodega'] = pd.to_datetime(etas['Fecha Arribo Bodega'], errors='coerce')
etas['Retiro puerto'] = pd.to_datetime(etas['Retiro puerto'], errors='coerce')
etas['Manifiesto'] = pd.to_datetime(etas['Manifiesto'], errors='coerce')
etas['ETA inicial'] = pd.to_datetime(etas['ETA inicial'], errors='coerce')

# 2. Apply the selection logic using .notna()
#    This is much cleaner and more reliable than comparing to 0.
etas['Fecha consolidada'] = np.where(
    etas['Fecha Arribo Bodega'].notna(),
    etas['Fecha Arribo Bodega'],
    np.where(
        etas['Retiro puerto'].notna(),
        etas['Retiro puerto'],
        np.where(
            etas['Manifiesto'].notna(),
            etas['Manifiesto'],
            etas['ETA inicial']
        )
    )
)

In [10]:
etas['Fecha consolidada'].value_counts().sort_index()

Fecha consolidada
2025-09-04    2
2025-09-06    2
2025-09-08    5
2025-09-10    1
2025-09-11    2
2025-09-12    3
2025-09-15    1
2025-09-16    2
2025-10-01    1
2025-10-15    1
Name: count, dtype: int64

In [13]:
tr['Nº.de.referencia'] = tr['Nº.de.referencia'].astype('str')

In [14]:


tr['Factura'] = np.where(
    tr['Status.línea.pedido'] == 'A',
    '872' + tr['Nº.de.referencia'],
    "0"
)

In [18]:
etas.drop_duplicates(subset='Factura BMW', inplace=True)

In [ ]:
tr_etas = tr.merge(etas[['Factura BMW','Fecha consolidada']], left_on='Factura', right_on='Factura BMW', how='left')

In [23]:
tr_etas.drop(columns=['Factura BMW'], inplace=True)

In [27]:
tr_etas['Fecha A'] = np.where(
    tr_etas['Status.línea.pedido']=='A',
    tr_etas['Fecha consolidada'],
    pd.NaT
    
)

In [29]:
tr_etas['Status'] = np.where(
    tr_etas['Status.línea.pedido'] == 'O',
    np.where(
        tr['Fecha.creación'] + pd.to_timedelta(tr['LT Fact'], unit='D') >= pd.to_datetime(hoy),"Transito","Transito Vencido"
    ),
    np.where(tr_etas['Status.línea.pedido']=="A",
             np.where(
                  tr_etas['Fecha A']  >= pd.to_datetime(hoy),"Facturado", "Facturado Vencido"
             ),"0") 
)

In [30]:
tr_etas['Status'].value_counts()

Status
0                    40200
Facturado Vencido     2308
Transito Vencido       702
Name: count, dtype: int64

In [39]:
tr_etas['Fecha Final'] = np.where(
    (tr_etas['Status.línea.pedido']=="A") & (tr_etas['Tipo.pedido']=='S') & (tr_etas['Status'] == 'Facturado Vencido') , (pd.to_datetime(hoy)+ pd.to_timedelta(45)),
            np.where(
                (tr_etas['Status.línea.pedido']=="A") & (tr_etas['Tipo.pedido']=='U') & ( tr_etas['Status'] == 'Facturado Vencido') , (pd.to_datetime(hoy)+ pd.to_timedelta(15)),
                np.where(
                    (tr_etas['Status.línea.pedido']=="A") & (tr_etas['Tipo.pedido']=='V') & (tr_etas['Status'] == 'Facturado Vencido') , (pd.to_datetime(hoy)+ pd.to_timedelta(5)),
                    np.where(
                        tr_etas['Status.línea.pedido']=="A", tr_etas['Fecha A'],
                        np.where(
                            tr_etas['Status.línea.pedido']=="O", tr_etas['Fecha consolidada'], pd.NaT
                        )
                    )
                        )

                    )
    
            )


In [41]:
# Define the conditions and corresponding choices
conditions = [
    (tr_etas['Status.línea.pedido'] == 'A') & (tr_etas['Tipo.pedido'] == 'S') & (tr_etas['Status'] == 'Facturado Vencido'),
    (tr_etas['Status.línea.pedido'] == 'A') & (tr_etas['Tipo.pedido'] == 'U') & (tr_etas['Status'] == 'Facturado Vencido'),
    (tr_etas['Status.línea.pedido'] == 'A') & (tr_etas['Tipo.pedido'] == 'V') & (tr_etas['Status'] == 'Facturado Vencido'),
    (tr_etas['Status.línea.pedido'] == 'A'),
    (tr_etas['Status.línea.pedido'] == 'O')
]

# Define the choices that correspond to each condition
choices = [
    pd.to_datetime('today').normalize() + pd.Timedelta(days=45),
    pd.to_datetime('today').normalize() + pd.Timedelta(days=15),
    pd.to_datetime('today').normalize() + pd.Timedelta(days=5),
    tr_etas['Fecha A'],
    tr_etas['Fecha O']
]

# Apply the conditions using np.select()
tr_etas['Resultado'] = np.select(conditions, choices, default=0)

In [43]:
tr_etas['Resultado'] = pd.to_datetime(tr_etas['Resultado'])

In [45]:
tr_etas['Resultado'].value_counts()

Resultado
1970-01-01 00:00:00.000000    40200
2025-11-09 00:00:00.000000     1948
2025-11-01 21:22:54.260027      465
2025-10-10 00:00:00.000000      338
2026-02-07 09:22:54.260027      221
2025-09-30 00:00:00.000000       22
2025-10-17 21:22:54.260027       15
2025-09-25 09:22:54.260027        1
Name: count, dtype: int64

In [48]:
condicion = [tr_etas['Status.línea.pedido']=='A', tr_etas['Status.línea.pedido']=='O']
opcion = [tr_etas['Cantidad.aconsejada'], tr_etas['Cantidad.requerida']]

tr_etas['Cantidad Final'] = np.select(condicion, opcion, 0)